 # rag 全流程

 - 创建向量数据库
 - 加载文档并解析成文档片段
 - 转化文档片段为向量
 - 把文档片段向量插入到向量数据库中
 - 创建检索器
 - 输入问题，转化成向量，聊天模式（跟上下文的关系）
 - 获取最相似的文档，输出

环境初始化

In [12]:
# load env
from dotenv import load_dotenv 
import os 
load_dotenv()

OLLAMA_API_URL = os.getenv("OLLAMA_API_URL")
MODEL_NAME = "llama3:8b"  # 使用的模型名称
EMBEDDING_MODEL = "bge-m3"  # 用于生成嵌入的模型
DB_PATH = "db"
DOCS_PATH = "./docs"

print(OLLAMA_API_URL)

http://home.chrissong.top:11434


创建向量模型

In [6]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL,base_url=OLLAMA_API_URL)

print(DB_PATH)

db


创建向量数据库

In [7]:
from langchain_community.vectorstores import Chroma

vector_store = Chroma(
        persist_directory= DB_PATH,
        embedding_function=embeddings,
        collection_name="document_collection",
    )

文档分割

In [18]:
from langchain_community.document_loaders import PDFPlumberLoader ,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

if vector_store._collection.count() == 0:
    print("未发现文档，开始加载 documents 目录下的文件...")
    directory_loader = DirectoryLoader(DOCS_PATH, glob="*.pdf", loader_cls=PDFPlumberLoader)
    documents = directory_loader.load()
    print(f"加载了 {len(documents)} 个文档")

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    split_docs = text_splitter.split_documents(documents)
    # 添加到向量库
    vector_store.add_documents(split_docs)
    print(f"成功加载 {len(split_docs)} 个文档片段到向量库")

创建大语言模型

In [14]:
from langchain_community.chat_models import ChatOllama

# 大语言模型
llm = ChatOllama(model=MODEL_NAME, base_url=OLLAMA_API_URL, temperature=0.7)

# 检索器
retriever = vector_store.as_retriever(search_kwargs={"k": 3})



创建rag chain

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_template("""
    请根据提供的上下文信息回答用户的问题。
    仅使用上下文内容，信息不足时请说明无法回答。

    上下文:
    {context}

    问题:
    {input}

    回答:
    """)
    
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

问答

In [ ]:
while True:
    question = input("\n请输入问题（输入q退出）: ")
    if question.lower() == 'q':
        break
    response = retrieval_chain.invoke({"input": question})
    
    print(f"\n问题: {question}")
    print(f"\n回答: {response['answer']}")
    
    print("\n相关文档片段:")
    for i, doc in enumerate(response["context"]):
        print(f"片段 {i+1}: {doc.page_content[:150]}...")